In [ ]:
#upload files to your colab environment
from google.colab import files
uploaded = files.upload()

Saving recipes.json.zip to recipes.json (1).zip


In [ ]:
!unzip 'recipes.json.zip'

Archive:  recipes.json.zip
replace recipes.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: [y]
error:  invalid response [[y]]
replace recipes.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: recipes.json            


In [ ]:
import torch
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                        Trainer, TrainingArguments

In [ ]:
!nvidia-smi

Fri Jan  5 04:48:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


with open('recipes.json') as f:
    data = json.load(f)

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts['Instructions']).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data,test_size=0.15)


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 10361
Test dataset length: 1829


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=9, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.evaluate()

{'eval_loss': 4.9390950202941895,
 'eval_runtime': 44.7103,
 'eval_samples_per_second': 103.108,
 'eval_steps_per_second': 1.633}

In [ ]:
trainer.train()

Step,Training Loss
500,3.383900
1000,2.218500
1500,1.947400
2000,1.806100
2500,1.713800
3000,1.643100
3500,1.584700
4000,1.547100
4500,1.506300
5000,1.483900


TrainOutput(global_step=7200, training_loss=1.7464277648925781, metrics={'train_runtime': 8140.0079, 'train_samples_per_second': 28.294, 'train_steps_per_second': 0.885, 'total_flos': 1.504454197248e+16, 'train_loss': 1.7464277648925781, 'epoch': 9.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.3765569925308228,
 'eval_runtime': 49.5889,
 'eval_samples_per_second': 92.964,
 'eval_steps_per_second': 1.472,
 'epoch': 9.0}

In [ ]:
trainer.save_model()

In [ ]:
from transformers import GPT2Config, pipeline
config = GPT2Config.from_pretrained('./gpt2-gerchef', max_length=800)

In [ ]:
import torch
# Ensure the model and data are on the same device (either GPU or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
chef = pipeline('text-generation', model=model, tokenizer=tokenizer, config=config, device=0 if device == "cuda" else -1)

In [ ]:
# from transformers import pipeline

# chef = pipeline('text-generation',model='./gpt2-gerchef', tokenizer='gpt2',config={'max_length':800})

In [ ]:
chef('Die Nudeln Kochen, Fleisch anbraten')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Die Nudeln Kochen, Fleisch anbraten bis es ein wenig angeröstet ist.  Das Suppengemüse (2 große Möhren, ein Stück Seller'}]

In [ ]:
chef('Die Nudeln Kochen')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Die Nudeln Kochen bringen, die Blätter hineinfüllen und 30 Minuten in den vorgeheizten Backofen schieben.Als Beilage schmecken getrocknet'}]

In [ ]:
chef('Zuerst Hähnchen')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Zuerst Hähnchenkeulen mit Senf bestreichen, mit Zitronenspalten garnieren und servieren.  Die Eier pochieren. Dafür in einem mittleren Top'}]

In [ ]:
chef('Der beste Weg, um einen Schokoladenkuchen zuzubereiten, ist')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Der beste Weg, um einen Schokoladenkuchen zuzubereiten, ist eigentlich einmal sehr lecker!Der könnten Speck braten man nur wen'}]